In [3]:
import numpy as np
import sympy as sp
import pylbm
import time
from ipywidgets import interact, interactive, Layout
%matplotlib inline

In [6]:
def runVortexSim(NReynolds, TiempoFinal, densidad):

    print('Cargando...')
    X, Y, LA = sp.symbols('X, Y, LA')
    rho, qx, qy = sp.symbols('rho, qx, qy')
    def bc_in(f, m, x, y):
        m[qx] = rhoo * v0

    def vorticity(sol):
        ux = sol.m[qx] / sol.m[rho]
        uy = sol.m[qy] / sol.m[rho]
        V = np.abs(uy[2:,1:-1] - uy[0:-2,1:-1] - ux[1:-1,2:] + ux[1:-1,0:-2])/(2*sol.domain.dx)
        return -V

    rayon = 0.05
    Re = NReynolds
    dx = 1./128 # spatial step
    la = 1. # velocity of the scheme
    Tf = TiempoFinal # final time of the simulation
    v0 = la/20 # maximal velocity obtained in the middle of the channel
    rhoo = densidad # mean value of the density
    mu = 1.e-3 # bulk viscosity
    eta = rhoo*v0*2*rayon/Re # shear viscosity

    # initialization
    xmin, xmax, ymin, ymax = 0., 3., 0., 1.
    dummy = 3.0/(la*rhoo*dx)
    s_mu = 1.0/(0.5+mu*dummy)
    s_eta = 1.0/(0.5+eta*dummy)
    s_q = s_eta
    s_es = s_mu
    s = [0.,0.,0.,s_mu,s_es,s_q,s_q,s_eta,s_eta]
    dummy = 1./(LA**2*rhoo)
    qx2 = dummy*qx**2
    qy2 = dummy*qy**2
    q2 = qx2+qy2
    qxy = dummy*qx*qy


    dico = {
        'box': {'x': [xmin, xmax],
                'y': [ymin, ymax],
                'label': [0, 2, 0, 0]
                },
        'elements': [pylbm.Circle([.3, 0.5*(ymin+ymax)+dx], rayon, label=1)],
        'space_step': dx,
        'scheme_velocity': la,
        'parameters': {LA: la},
        'schemes': [
            {
                'velocities': list(range(9)),
                'conserved_moments': [rho, qx, qy],
                'polynomials': [
                    1, LA*X, LA*Y,
                    3*(X**2+Y**2)-4,
                    (9*(X**2+Y**2)**2-21*(X**2+Y**2)+8)/2,
                    3*X*(X**2+Y**2)-5*X, 3*Y*(X**2+Y**2)-5*Y,
                    X**2-Y**2, X*Y
                ],
                'relaxation_parameters': s,
                'equilibrium': [
                    rho, qx, qy,
                    -2*rho + 3*q2,
                    rho-3*q2,
                    -qx/LA, -qy/LA,
                    qx2-qy2, qxy
                ],
            },
        ],
        'init': {rho:rhoo,
                qx:0.,
                qy:0.
        },
        'boundary_conditions': {
            0: {'method': {0: pylbm.bc.BouzidiBounceBack}, 'value': bc_in},
            1: {'method': {0: pylbm.bc.BouzidiBounceBack}},
            2: {'method': {0: pylbm.bc.NeumannX}},
        },
        'generator': 'cython',
    }
    print('Esto puede tardar unos minutos...')
    sol = pylbm.Simulation(dico)
    while sol.t < Tf:
        sol.one_time_step()

    print('Listo')
    viewer = pylbm.viewer.matplotlib_viewer
    fig = viewer.Fig(figsize=(12, 8))
    ax = fig[0]
    im = ax.image(vorticity(sol).transpose(), clim = [-3., 0])
    ax.ellipse([.3/dx, 0.5*(ymin+ymax)/dx], [rayon/dx,rayon/dx], 'r')
    ax.title = 'Calle de Vortice de Von Karman: t = {:.0f} [s], Re = {:.0f}'.format(sol.t, Re)
    fig.show()

In [ ]:
interactive_plot = interactive(runVortexSim, NReynolds=(1, 8000, 10), TiempoFinal=(0.1, 100, 1), densidad=(0.1, 3, 0.1))

# interactive_plot.children[0].value = 1000;
# interactive_plot.children[1].value = 65;
# interactive_plot.children[2].value = 1;

interactive_plot.children[0].layout = Layout(width='500px')
interactive_plot.children[1].layout = Layout(width='500px')
interactive_plot.children[2].layout = Layout(width='500px')


# interactive_plot.children[0].description = 'Numero de Reynolds';

output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

interactive(children=(IntSlider(value=3991, description='NReynolds', layout=Layout(width='500px'), max=8000, m…